In [1]:
! pip install pydub

In [12]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine, Square, Sawtooth
import torch
import colorsys

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [14]:
df = pd.read_csv('/content/galaxy_image_pixel_data.csv')  # Replace with your CSV file path

# Take the first 1000 rows
df_subset = df.head(1000)

In [15]:
# Create an empty audio segment
audio = AudioSegment.silent(duration=0)

### Function to calculate brightness from RGB

In [16]:
def brightness(r, g, b):
    return 0.299 * r + 0.587 * g + 0.114 * b

### Function to map brightness to pitch

In [17]:
def brightness_to_pitch(brightness):
    # Map brightness (0-255) to frequency (220-880 Hz)
    return 220 + (brightness / 255.0) * 660

In [18]:
def rgb_to_sound(r, g, b, duration=100):
    pitch = brightness_to_pitch(brightness(r, g, b))

    # Choose sound based on dominant color
    if r > g and r > b:
        tone = Sine(pitch).to_audio_segment(duration=duration)
    elif g > r and g > b:
        tone = Square(pitch).to_audio_segment(duration=duration)
    else:
        tone = Sawtooth(pitch).to_audio_segment(duration=duration)

    return tone

In [19]:
r_tensor = torch.tensor(df_subset['R'].values, dtype=torch.float32).to(device)
g_tensor = torch.tensor(df_subset['G'].values, dtype=torch.float32).to(device)
b_tensor = torch.tensor(df_subset['B'].values, dtype=torch.float32).to(device)

## Generate sound for each pixel in the subset

In [20]:
for r, g, b in zip(r_tensor, g_tensor, b_tensor):
    tone = rgb_to_sound(r.item(), g.item(), b.item())
    audio += tone

## Export the final sound file

In [21]:
output_filename = "output_sound_1000_rows_pitch_mapping.wav"
audio.export(output_filename, format="wav")
print(f"Generated {output_filename}")

Generated output_sound_1000_rows_pitch_mapping.wav
